# SDialog Tutorial: Building Agents with Thinking and Tools

<p align="right" style="margin-right: 8px;">
    <a target="_blank" href="https://colab.research.google.com/github/idiap/sdialog/blob/main/tutorials/7.agents_with_tools_and_thoughts.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p>

A step-by-step guide to create conversational agents in SDialog, enable hidden reasoning, and add tool calling capabilities.

## Introduction
This tutorial walks you through creating practical SDialog agents, progressively adding hidden reasoning (think) and tool-calling capabilities.

What you will learn:
- Enable hidden thinking for better reasoning
- Register and call tools (functions) from an agent
- Inspect conversations and events


In [1]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
from IPython import get_ipython


if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    # Installing Ollama (if you are not planning to use Ollama, you can just comment these lines to speed up the installation)
    !curl -fsSL https://ollama.com/install.sh | sh

    # Installing sdialog
    !git clone https://github.com/idiap/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    import os
    get_ipython().system = os.system

Running in Jupyter Notebook


### 1.2 Start Ollama server (optional)
If you plan to use an Ollama backend locally, start the server in the background. If you use OpenAI/Bedrock/HF instead, you can skip this cell.

In [2]:
# Let's run the ollama server in the background
!OLLAMA_KEEP_ALIVE=-1 ollama serve > /dev/null 2>&1 &
!sleep 5

0

### 1.3 Configure the LLM backend
Set the global model used by SDialog. You can select any backend via the "BACKEND:MODEL" connection string, for example:
- "openai:gpt-4.1"
- "ollama:gemma3:27b"
- "amazon:anthropic.claude-3-5-sonnet-20240620-v1:0"
- "huggingface:HuggingFaceTB/SmolLM-360M"

Notes:
- Make sure the required credentials are available as environment variables for cloud providers.
- We will use a Qwen3 model that supports tool use natively.

In [ ]:
import sdialog

# sdialog.config.llm("ollama:qwen3:30b")
sdialog.config.llm("ollama:qwen3:14b")

## 2. Vanilla Agents (no thinking, no tools)
We first build two basic agents (a customer and a support agent) and let them talk without hidden reasoning or tools.

### 2.1 Generate example personas
We will generate a realistic customer persona and a support agent persona using PersonaGenerator.

In [ ]:
from sdialog.generators import PersonaGenerator
from sdialog.personas import Customer

customer_generator = PersonaGenerator(Customer(customer_id="12345",
                                               issue="Modem Viper 3000 stopped working",
                                               anger_level="high",
                                               times_called=3))
customer_persona = customer_generator.generate()

customer_persona.print()

[2025-09-12 18:24:05] INFO:sdialog.util:Loading Ollama model: qwen3:14b


[version] 0.1.0+fdf16e10134b2ac7dea23518390cf5b09aa10380
[timestamp] 2025-09-12T16:24:16Z
[model] {'name': 'ollama:qwen3:14b', 'seed': 3546130221, 'repeat_penalty': 1, 'temperature': 0.6, 'top_k': 20, 'top_p': 0.95}
[seed] 3546130221
[id] f3b42567-ab25-40ef-869a-accde55b4948
[className] Customer
--- Customer Begins ---
[Name] James Carter
[Age] 42
[Gender] Male
[Language] English
[Customer id] 12345
[Occupation] IT Manager
[Account tenure] 3 years
[Membership level] Platinum
[Loyalty status] Loyal
[Fidelity score] 92
[Issue] Modem Viper 3000 stopped working
[Issue category] Technical
[Issue description] The modem has stopped connecting to the internet. It was working fine until yesterday, and no changes were made to the network setup.
[Issue history] Previous issues included a router malfunction 6 months ago and a slow internet connection 1 year ago.
[Desired outcome] A working modem and a replacement if necessary
[Knowledge domain] Basic networking concepts
[Technical expertise] Moder

In [ ]:
from sdialog.generators import PersonaGenerator
from sdialog.personas import SupportAgent

support_generator = PersonaGenerator(SupportAgent(agent_id="A123",
                                                  politeness="high",
                                                  rules=("Make sure to always let the customer know the next actions you are taking, "
                                                         "and then gather all the required information required to run each function. "
                                                         "Make sure to verify the identity before proceeding.")))

support_persona = support_generator.generate()

support_persona.print()

[2025-09-12 18:24:16] INFO:sdialog.util:Loading Ollama model: qwen3:14b


[version] 0.1.0+fdf16e10134b2ac7dea23518390cf5b09aa10380
[timestamp] 2025-09-12T16:24:20Z
[model] {'name': 'ollama:qwen3:14b', 'seed': 1906005053, 'repeat_penalty': 1, 'temperature': 0.6, 'top_k': 20, 'top_p': 0.95}
[seed] 1906005053
[id] 830df7ce-5aa2-4788-a72c-7a6bcbc099a2
[className] SupportAgent
--- SupportAgent Begins ---
[Name] Jordan Lee
[Language] English
[Agent id] A123
[Role] Customer Support Agent
[Experience years] 5 years
[Product scope] Software subscriptions, account management, billing issues
[Product knowledge level] expert
[Communication style] empathetic and clear
[Empathy level] high
[Politeness] high
[Resolution authority level] can resolve most issues independently
[Escalation policy] Escalate to senior agent if issue involves legal concerns, unresolved billing disputes, or high-severity technical failures
[Average handle time] 6m
[Adherence notes] Always follow the step-by-step resolution process and document all customer interactions
[Stress tolerance] high, rem

### 2.2 Create agents from personas
Instantiate two Agent objects: one for the Support Agent and one for the Customer. The customer starts the conversation with "Hello, good afternoon" as the first utterance.

In [ ]:
from sdialog.agents import Agent

support_agent = Agent(support_persona,
                      name="AGENT")
customer = Agent(customer_persona,
                 first_utterance="Hello, good afternoon",
                 name="CUSTOMER")

[2025-09-12 18:24:20] INFO:sdialog.util:Loading Ollama model: qwen3:14b
[2025-09-12 18:24:20] INFO:sdialog.util:Loading Ollama model: qwen3:14b


In [7]:
dialog = customer.talk_with(support_agent, max_turns=5)
dialog.print(think=True)

[dialog_id] 0c377ebf-67e3-44c3-8d9f-42d73a726042
[model] {'name': 'ollama:qwen3:14b', 'seed': 13, 'temperature': 0.6, 'top_k': 20, 'top_p': 0.95, 'repeat_penalty': 1}
[seed] 2789801852
--- Dialogue Begins ---
[CUSTOMER] Hello, good afternoon
[AGENT] Hello! How can I assist you today?
[CUSTOMER] I'm calling regarding my modem, the Viper 3000 – it's completely stopped working, and I need this resolved immediately.
[AGENT] I'm sorry to hear about the issue with your Viper 3000 modem. Let me start by verifying your account details so I can look up your service information. Could you please provide your account number or the phone number associated with your service?
[CUSTOMER] I’ve already provided my account number earlier – it’s 12345. I’m a platinum member, so I expect a swift resolution. Let’s not waste time.
[AGENT] Thank you for providing your account number, and I apologize for the inconvenience. Let me check the status of your modem and see what steps I can take to resolve the issu

## 3. Enable Hidden Thinking
Hidden thinking (think=True) lets the agent plan before answering. We will create a thinking agent and compare outputs.

In [ ]:
support_agent_tools = Agent(support_persona,
                            think=True,  # Enable thinking!
                            name="AGENT")

[2025-09-12 18:24:26] INFO:sdialog.util:Loading Ollama model: qwen3:14b


In [9]:
dialog = customer.talk_with(support_agent_tools, max_turns=5)
dialog.print()

[dialog_id] 0ce2cec4-2c4e-417a-a712-e150c5f3b504
[model] {'name': 'ollama:qwen3:14b', 'seed': 13, 'temperature': 0.6, 'top_k': 20, 'top_p': 0.95, 'repeat_penalty': 1}
[seed] 2569826378
--- Dialogue Begins ---
[CUSTOMER] Hello, good afternoon
[AGENT] Good afternoon! How can I assist you today?
[CUSTOMER] I'm calling regarding my modem, the Viper 3000. It stopped working yesterday, and I need this resolved immediately.
[AGENT] I understand this is urgent. First, may I verify your account details to ensure I can assist you directly? Once confirmed, I'll guide you through troubleshooting steps for the Viper 3000.
[CUSTOMER] I'm already a platinum member with customer ID 12345. I don't have time for verification steps—I need a resolution now. This is my third call this week, and I'm not going through the same process again.
[AGENT] I apologize for the inconvenience, Mr. Lee. Let me confirm your identity using your platinum membership and customer ID 12345. One moment, please.
--- Dialogue E

In [10]:
dialog.print(think=True)

[dialog_id] 0ce2cec4-2c4e-417a-a712-e150c5f3b504
[model] {'name': 'ollama:qwen3:14b', 'seed': 13, 'temperature': 0.6, 'top_k': 20, 'top_p': 0.95, 'repeat_penalty': 1}
[seed] 2569826378
--- Dialogue Begins ---
[CUSTOMER] Hello, good afternoon
[AGENT] (thinking) Okay, the user said "Hello, good afternoon." I need to respond as Jordan Lee, the customer support agent. First, I should acknowledge their greeting politely. Since Jordan's communication style is empathetic and clear, I should make sure to be friendly and offer assistance. The response should be concise, as per the guidelines. Let me check the rules again: responses should be one utterance, no multiple questions. So, a simple greeting back and offer to help. Also, need to remember to verify identity later, but not right away. Maybe start with "Good afternoon! How can I assist you today?" That's friendly and opens the door for them to explain their issue. No need to ask multiple questions at once. Yep, that fits. 
[AGENT] Good af

## 4. Agents with Tools
Tools allow an agent to call functions (queries/actions) to augment capabilities. We'll define tools, attach them to the agent, and inspect tool calls.

### 4.1 Define tools for the Support Agent
We define three groups of (mock up) functions the agent can call:
- RAG-like retrieval: get_product_documentation
- Query functions: verify_account, check_modem_status, get_product_warranty_info
- Action functions: ship_replacement_modem, schedule_technician, escalate_ticket

In [ ]:
# Simulates a RAG-like tool available for the support agent
def get_product_documentation(product: str, model: str) -> dict:
    """Retrieve product documentation for a specific product and model.
    Args:
        product: The product name.
        model: The product model.
    Returns:
        JSON with a list of documentation snippets.
    """
    docs = [
        f"{product} {model} is a high-performance modem designed for home and office use.",
        f"It supports speeds up to 1 Gbps and is compatible with most ISPs.",
        f"The {model} features dual-band Wi-Fi, multiple Ethernet ports, and advanced security features.",
        f"To reset the {model}, press and hold the reset button for 10 seconds.",
        f"For troubleshooting connectivity issues, ensure that all cables are securely connected and restart the modem.",
    ]
    return {"documentation": docs}

In [ ]:
# Examples of query functions that the support agent can call
from typing import Optional

def get_product_warranty_info(product: str, model: str) -> dict:
    """Retrieve warranty and coverage information for a specific product and model.
    Args:
        product: The product name.
        model: The product model.
    Returns:
        JSON with warranty status and coverage details.
    """
    return {
        "warranty_status": "active",
        "coverage": {
            "accidental_damage": True,
            "theft_protection": False,
            "technical_support": True,
        },
    }


def verify_account(customer_id: str) -> dict:
    """Verify customer account details and status.
    Args:
        customer_id: The customer's unique id.
    Returns:
        JSON with customer id and existence flag.
    """
    return {"customer_id": customer_id, "exists": True}

def check_modem_status(
    customer_id: str, model: str = "Viper 3000", serial: Optional[str] = None
) -> dict:
    """Query last known status of customer's modem.
    Args:
        customer_id: Customer id.
        model: Modem model name.
        serial: Optional device serial in case the customer has more than 1 modem.
    Returns:
        JSON with device status and diagnostics.
    """
    base = {
        "customer_id": customer_id,
        "model": model,
        "serial": serial or "VP3K-XYZ-0001",
        "firmware": "v1.8.2",
        "last_online": "two days ago",
        "power_state": "no_power",
        "wan_link": "down",
        "lan_ports": 0,
        "wifi": "down",
        "diagnostic_note": "No heartbeat for >48h; likely power supply failure or dead device.",
    }
    return base


In [ ]:
# Example of action functions that the support agent can call (these execute real actions in a real system)
def ship_replacement_modem(customer_id: str, model: str, address: str) -> dict:
    """Create a shipment order for a replacement modem.
    Args:
        customer_id: Customer id.
        model: Desired model to ship.
        address: Address for the shipment.
    Returns:
        JSON with order id and ETA.
    """
    return {"status": "scheduled", "order_id": "ORD-12345", "priority": "high", "eta_days": 1}


def schedule_technician(
    customer_id: str, window: str = "Tomorrow 09:00-12:00", priority: str = "High", instructions: str = None, address: str = None
) -> dict:
    """Book an on-site technician visit.
    Args:
        customer_id: Customer id.
        window: Preferred time window (local time).
        priority: Scheduling priority.
        instructions: Special instructions for the technician.
        address: Address for the technician visit.
    Returns:
        JSON with appointment id and status.
    """
    return {"status": "scheduled", "appointment_id": "APPT-67890"}


def escalate_ticket(customer_id: str, severity: str = "P1", notes: str = "") -> dict:
    """Open or escalate a support ticket.
    Args:
        customer_id: Customer id.
        severity: P1..P4 severity code.
        notes: Free-form context.
    Returns:
        JSON with ticket id and status.
    """
    return {"status": "done", "ticket_id": "TICK-54321"}

### 4.2 Build an agent with tools
Attach the defined functions to our Support Agent and enable think=True so it can better decide when to call the tools.

In [14]:
support_agent_tools = Agent(support_persona,
                         think=True,
                         tools=[
                            # queries
                            verify_account,
                            check_modem_status,
                            get_product_documentation,
                            get_product_warranty_info,
                            # actions
                            ship_replacement_modem,
                            schedule_technician,
                            escalate_ticket,
                         ],
                         name="AGENT")

[2025-09-12 18:24:46] INFO:sdialog.util:Loading Ollama model: qwen3:14b


In [ ]:
support_agent_tools.reset(seed=42)

query = "Hello, I'm Michael and want to know the maximum speed supported by the Viper 3000 modem"
print(support_agent_tools(query))

The Viper 3000 modem supports maximum speeds of up to **1 Gbps**. Let me know if you need further details!


In [ ]:
support_agent_tools.reset(seed=42)

support_agent_tools(query, return_events=True)

[Event(agent='AGENT', action='think', actionLabel=None, content='Okay, let\'s see. The user is asking about the maximum speed supported by the Viper 3000 modem. First, I need to figure out which function to use. The available functions include get_product_documentation and get_product_warranty_info. Since the user is asking about product specifications, get_product_documentation seems appropriate. The parameters required are product and model. The product here is "modem" and the model is "Viper 3000". So I should call get_product_documentation with those parameters. I need to make sure I use the correct function name and structure the arguments properly. Let me double-check the function\'s description to confirm. Yes, it retrieves documentation snippets for a product and model. That should include the speed information. Alright, time to format the tool call correctly.\n', timestamp=1757694297),
 Event(agent='AGENT', action='tool', actionLabel='call', content={'name': 'get_product_docum

In [19]:
dialog = customer.talk_with(support_agent_tools, max_turns=10)
dialog.print()

[dialog_id] 14b7c525-5474-4a8f-8fb0-f541929f07c5
[model] {'name': 'ollama:qwen3:14b', 'seed': 13, 'temperature': 0.6, 'top_k': 20, 'top_p': 0.95, 'repeat_penalty': 1}
[seed] 2007304206
--- Dialogue Begins ---
[CUSTOMER] Hello, good afternoon
[AGENT] Good afternoon! How can I assist you today?
[CUSTOMER] I'm calling regarding my modem, the Viper 3000—it's completely stopped working, and I need this resolved immediately.
[AGENT] I'll need to verify your account and check your modem's status. Could you please provide your customer ID and the modem's serial number?
[CUSTOMER] My customer ID is 12345, and the modem's serial number is V3000-123456789. I've already mentioned this is a Platinum account, so please expedite the process.
[AGENT] I see the modem is not receiving power, and diagnostics indicate a potential hardware failure. Since this is a Platinum account, I can expedite a replacement. Could you share your shipping address so I can arrange immediate shipment of a new Viper 3000 mo

In [20]:
dialog.print(all=True)

[dialog_id] 14b7c525-5474-4a8f-8fb0-f541929f07c5
[model] {'name': 'ollama:qwen3:14b', 'seed': 13, 'temperature': 0.6, 'top_k': 20, 'top_p': 0.95, 'repeat_penalty': 1}
[seed] 2007304206
--- Dialogue Begins ---
[CUSTOMER] Hello, good afternoon
[AGENT] (thinking) Okay, the user said "Hello, good afternoon." I need to respond as Jordan Lee, the customer support agent. First, I should greet them back politely. Since the communication style is empathetic and clear, I should make sure to be friendly and offer assistance. Let me check the guidelines—responses should be concise, one utterance. So, something like, "Good afternoon! How can I assist you today?" That's polite, opens the door for them to explain their issue, and follows the persona's traits. No need to ask multiple questions at once. Just a simple, welcoming response. 
[AGENT] Good afternoon! How can I assist you today?
[CUSTOMER] I'm calling regarding my modem, the Viper 3000—it's completely stopped working, and I need this resolve